In [96]:
import pandas as pd
import unidecode
import string
import sklearn.feature_extraction
import itertools
import re

In [118]:
rm_punc = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS

In [126]:
tweets = pd.read_json('/Users/tomashegewisch/Downloads/030110/test.json', lines=True, orient='record')

In [112]:
tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'mentions', 'urls',
       'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near', 'geo',
       'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [174]:
tweets['tweet']

0        girls who go to wits uni take 'phuza thursdays...
1                                       Condolences man❤️🙏
2                                   Qala adle bese uyalala
3        Make sure you use this finger I heard it does ...
4        My boy 😍 https://twitter.com/MabokoItumeleng/s...
                               ...                        
19725                     Ke 2😩 pic.twitter.com/WLNvFMu9Cx
19726                                                    😂
19727                I’d finish with unisa from my bedroom
19728    If Cardi can eat a chicken wing with "hairs & ...
19729    Mmmmm. Iyashisa! 🎉\n\nTake the #MmmzansiFlavou...
Name: tweet, Length: 19725, dtype: object

In [114]:
def clean_tweet_word(word):
    if (len(word) > 1) and (word[0] == '#' or word[0] == '@'):
        return word[0] + word[1:].translate(rm_punc).strip()
    if word.startswith('http') or word.startswith('pic.twitter.com'):
        return ''
    if word in stop_words:
        return ''
    return word.translate(rm_punc).strip()

def clean_tweet(tweet):
    tweet = unidecode.unidecode(tweet).lower().split()
    tweet = [clean_tweet_word(x) for x in tweet]
    tweet = list(itertools.chain.from_iterable([x.split() for x in tweet if x != '']))
    tweet = [x for x in tweet if len(x) > 1]
    return tweet

In [175]:
tokenized_tweets = tweets['tweet'].apply(clean_tweet)

In [116]:
print(tweets['tweet'])

0        girls who go to wits uni take 'phuza thursdays...
1                                       Condolences man❤️🙏
2                                   Qala adle bese uyalala
3        Make sure you use this finger I heard it does ...
4        My boy 😍 https://twitter.com/MabokoItumeleng/s...
                               ...                        
19725                     Ke 2😩 pic.twitter.com/WLNvFMu9Cx
19726                                                    😂
19727                I’d finish with unisa from my bedroom
19728    If Cardi can eat a chicken wing with "hairs & ...
19729    Mmmmm. Iyashisa! 🎉\n\nTake the #MmmzansiFlavou...
Name: tweet, Length: 19730, dtype: object


In [127]:
print("BEFORE")
print(len(tweets))
tweets.drop_duplicates(subset=['id'], keep="first", inplace=True)
print("\nAFTER")
print(len(tweets))

BEFORE
19730

AFTER
19725


In [161]:
print(tweets['tweet'][150:250])

tweets['urls'][17]

155    Still not sorted days later and my daughter do...
156    @zimiphib1 sebephakathi 🏊🏽‍♀️ esizibeni nampa:...
157    @Zimiphib1 #sibhukudakwesimanzozo ngithatha Ph...
158                                              Aich 🔥🔥
159    I wanna go back to being my parent's Kontso😢th...
                             ...                        
250                            This one can go to hell .
251    No more fist pumps when we playing pic.twitter...
252                                       Aninamali nje.
253    @kingmathibela 😭 https://twitter.com/linc_smas...
254                                Braaaaaaaaaaaa!!!!!!!
Name: tweet, Length: 100, dtype: object


['https://twitter.com/CulpritEnter/status/1235634589638500358']

In [10]:
tfidf_transform = sklearn.feature_extraction.text.TfidfVectorizer(analyzer=clean_tweet, min_df=0.001)
X = tfidf_transform.fit_transform(tweets['tweet'])

In [59]:
tfidf_transform

TfidfVectorizer(analyzer=<function clean_tweet at 0x7ffa40f8f790>, min_df=0.001)

In [69]:
# https://www.geeksforgeeks.org/python-check-url-string/
#GrOss Regex 
def find(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url]
find("I love to use this website when i am working with code https://www.geeksforgeeks.org/python-check-url-string/")
#twitter compressing the URL in their Tweets...
tweet_urls_tiny = []
for i in tweets['tweet'].values.tolist():
    temp = find(i)
    if temp != []:
        for i in temp:
            tweet_urls_tiny.append(i)
# THIS IS NOT NEEDED AS TWITTER STORES THE DOMAINS IN A FEILD>>>
find("jhkwqjdhkqjwhdqwdjhqwd jqlwdkjqlwkjd https://www.geeksforgeeks.org/python-check-url-string/ gldglqwdhgqwdqwjdhjwhdqjwhdgqjwhdgqhjwdgqdhjwgjqhwd")

['https://www.geeksforgeeks.org/python-check-url-string/']

In [164]:
def phone_number(text):
    regex = r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}"
    re.findall(regex, string)
phone_number("this is cool 021 9979544")


reg = re.compile(".*?(\(?\d{3})? ?[\.-]? ?\d{3} ?[\.-]? ?\d{4}).*?", re.S)

error: missing ), unterminated subpattern at position 0

In [170]:
from langdetect import detect
print(detect("War doesn't show who's right, just who's left."))
print(detect("Sesi condomize kangaka,sesizobulawa ukuthimula"))
print(detect("Xa kanye ndizo charger iphone... wemka umbane. Oh Well.."))
print(detect("Saul Barnard is ‘n houtkapper met ‘n rustelose siel"))
print(detect("uBhuti Mongezi ebezothi  Uxov\'ebhaka https://twitter.com/Magleesta/status/1237099631156375556\xa0…"))
print(detect("@Betusile_Mcinga ansbwl ingoma esonwaba ngayo ingoma yomnikelo baba #lit pic.twitter.com/AvrZJ0GvZX"))
print(detect("Uba bekusemelika u Phindile ngeyene offer yokusebenzela i CIA kodwaaa lilizwe lika Cupcake eli😌"))

en
sw
sw
af
sw
tl
sw


In [173]:
for i in tweets['tweet']:
    print(i)
    if detect(i) != "en":
        tweets.drop([i])
        print(i)

girls who go to wits uni take 'phuza thursdays' seriously. – at Stones
Condolences man❤️🙏


KeyError: "['Condolences man❤️🙏'] not found in axis"